## Grid.ipynb, Ankit Upadhyay

In [1]:
import numpy as np 
import cv2
import random

In [2]:
img_path="shape.jpeg"

In [3]:
image =cv2.imread(img_path)

In [4]:
image = cv2.GaussianBlur(image, (5, 5), 0)

In [5]:
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [6]:
edges = cv2.Canny(gray_image, 50, 150, apertureSize=3)

In [7]:
_, mask = cv2.threshold(gray_image, 50, 255, cv2.THRESH_BINARY_INV)  


kernel = np.ones((5, 5), np.uint8)  
morphed_edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

In [8]:
lines = cv2.HoughLinesP(morphed_edges, 1, np.pi / 180, 50, minLineLength=20, maxLineGap=30)

In [9]:
len(lines)

17

In [10]:
def classify_lines(filtered_lines):
    vertical_lines = []
    horizontal_lines = []
    
    for line in filtered_lines:
        x1, y1, x2, y2 = line[0]
        
        
        if abs(x2 - x1) < 10:  
            vertical_lines.append(line)
        
        elif abs(y2 - y1) < 10:  
            horizontal_lines.append(line)
    
    return vertical_lines, horizontal_lines

In [11]:
def filter_close_lines(lines, orientation='vertical', threshold=10):
   
    if orientation == 'vertical':
        
        lines.sort(key=lambda line: line[0][0])
    elif orientation == 'horizontal':
        
        lines.sort(key=lambda line: line[0][1])
    
    filtered_lines = []
    
    i = 0
    while i < len(lines):
        largest_line = lines[i]
        x1, y1, x2, y2 = largest_line[0]
        
     
        j = i + 1
        while j < len(lines):
            x1_next, y1_next, x2_next, y2_next = lines[j][0]
            
            if orientation == 'vertical':
                
                if abs(x1_next - x1) < threshold:
                    
                    if abs(y2 - y1) < abs(y2_next - y1_next):  
                        largest_line = lines[j]
                else:
                    break
            elif orientation == 'horizontal':
             
                if abs(y1_next - y1) < threshold:
                    
                    if abs(x2 - x1) < abs(x2_next - x1_next): 
                        largest_line = lines[j]
                else:
                    break
            
            j += 1
        
        
        filtered_lines.append(largest_line)
        i = j  
    
    return filtered_lines

In [12]:
vertical_lines, horizontal_lines = classify_lines(lines)

In [18]:
vertical_lines
blank_image = np.ones_like(image) * 255  
for line in vertical_lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(blank_image, (x1, y1), (x2, y2), (0, 0, 255), 2)  
cv2.imshow('Filtered Lines', blank_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
blank_image = np.ones_like(image) * 255  
for line in horizontal_lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(blank_image, (x1, y1), (x2, y2), (0, 0, 255), 2) 
cv2.imshow('Filtered Lines', blank_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
filtered_vertical_lines = filter_close_lines(vertical_lines, orientation='vertical', threshold=20)
filtered_horizontal_lines = filter_close_lines(horizontal_lines, orientation='horizontal', threshold=20)


In [17]:
final_lines = filtered_vertical_lines + filtered_horizontal_lines

blank_image = np.ones_like(image) * 255  
for line in final_lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(blank_image, (x1, y1), (x2, y2), (0, 0, 255), 2)  

cv2.imshow('Filtered Lines', blank_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
final_lines

[array([[631, 364, 635, 118]], dtype=int32),
 array([[847, 301, 850, 121]], dtype=int32),
 array([[656, 114, 847, 120]], dtype=int32),
 array([[372, 354, 637, 358]], dtype=int32)]

In [20]:
copy_line=final_lines

In [21]:
hsv_image = cv2.imread(img_path)


hsv_image = cv2.cvtColor(hsv_image, cv2.COLOR_BGR2HSV)


lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

lower_green = np.array([40, 50, 50])
upper_green = np.array([90, 255, 255])


mask_red1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
mask_red2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
mask_red = mask_red1 + mask_red2 

mask_green = cv2.inRange(hsv_image, lower_green, upper_green)


contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_green, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


In [22]:

blank_image = np.ones_like(hsv_image) * 255
for contour in contours_red:
    M = cv2.moments(contour)
    if M['m00'] > 0:  
        cx_red = int(M['m10'] / M['m00'])
        cy_red = int(M['m01'] / M['m00'])
     
        cv2.circle(blank_image, (cx_red, cy_red), 5, (0, 0, 255), -1)  

for contour in contours_green:
    M = cv2.moments(contour)
    if M['m00'] > 0: 
        cx_green = int(M['m10'] / M['m00'])
        cy_green = int(M['m01'] / M['m00'])
       
        cv2.circle(blank_image, (cx_green, cy_green), 5, (0, 255, 0), -1) 


cv2.imshow('Detected Dots', blank_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
import numpy as np
import cv2

def distance(point1, point2):
  
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def find_closest_line(origin, lines):

    min_dist = float('inf')
    closest_line = None
    new_origin = None
    for line in lines:
        x1, y1, x2, y2 = line[0]
        start = (x1, y1)
        end = (x2, y2)

        dist_start = distance(origin, start)
        dist_end = distance(origin, end)

       
        if dist_start < dist_end:
            dist = dist_start
            point_to_set_as_origin = end  
            line_to_add = line  
            line_to_remove=line
            
        else:
            dist = dist_end
            point_to_set_as_origin = start 
            line_to_add = np.array([[x2, y2, x1, y1]])  
            line_to_remove=line
            
       
        if dist < min_dist:
            min_dist = dist
            closest_line = line_to_add
            new_origin = point_to_set_as_origin
            remove_line=line_to_remove

    return closest_line, new_origin,remove_line



In [24]:
def connect_lines_from_origin(origin, lines):

    ordered_lines = []
    remaining_lines = lines.copy()


    while remaining_lines:
 
        closest_line, new_origin,line_to_remove = find_closest_line(origin, remaining_lines)

        ordered_lines.append(closest_line)
    
        remaining_lines = [line for line in remaining_lines if not np.array_equal(line, line_to_remove)]
    
      
        origin = new_origin

    return ordered_lines



In [25]:
red_point = [cx_red, cy_red] 

ordered_lines = connect_lines_from_origin(red_point, copy_line)

In [26]:



blank_image = np.ones_like(image) * 255 
for line in ordered_lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(blank_image, (x1, y1), (x2, y2), (0, 0, 255), 2)  

cv2.imshow('Ordered Lines', blank_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

for idx, line in enumerate(ordered_lines):
    x1, y1, x2, y2 = line[0]
    print(f"Line {idx+1}: ({x1}, {y1}) -> ({x2}, {y2})")

Line 1: (847, 301) -> (850, 121)
Line 2: (847, 120) -> (656, 114)
Line 3: (635, 118) -> (631, 364)
Line 4: (637, 358) -> (372, 354)


In [27]:
def get_directions(ordered_lines):

    directions = []

    
    for i in range(len(ordered_lines)):
       
        x1, y1, x2, y2 = ordered_lines[i][0]

        start_point = (x1, y1)  
        next_point = (x2, y2)  

     
        dx = next_point[0] - start_point[0]
        dy = next_point[1] - start_point[1]

        
        if abs(dx) > abs(dy):  
            if dx > 0:
                directions.append('R')
            else:
                directions.append('L')
        else:  
            if dy > 0:
                directions.append('D')
            else:
                directions.append('U')

    return directions


directions = get_directions(ordered_lines)


print(directions)

['U', 'L', 'D', 'L']
